In [1]:
import pyrpe
import pygsti
from scipy.linalg import expm
import numpy as np
from pygsti.tools import unitary_to_superop
from pygsti.modelpacks import smq1Q_XYZI
from matplotlib import pyplot as plt
from pygsti.circuits import Circuit
from pyrpe.src.quapack.pyRPE import RobustPhaseEstimation
from pyrpe.src.quapack.pyRPE.quantum import Q

In [2]:
# to prepare |+> with Gxpi2 and Gzpi2, we apply Gzpi2 Gxpi2 Gzpi2^3
# this verifies  Gzpi2 Gxpi2 Gzpi2^3 == Gypi2
model = smq1Q_XYZI.target_model()
Gzpi2 = model['Gzpi2', 0].to_dense()
Gxpi2 = model['Gxpi2', 0].to_dense()
np.allclose(Gzpi2@Gxpi2@Gzpi2@Gzpi2@Gzpi2,  model['Gypi2', 0].to_dense())

True

In [3]:
def make_noisy_z_unitary(alpha):
    generator = -(1j/2) *(np.pi/2 + alpha)* pygsti.sigmaz
    return expm(generator)

def make_noisy_x_unitary(epsilon, theta):
    generator = -(1j/2) * (np.pi/2 + epsilon) * (np.cos(theta) * pygsti.sigmax + np.sin(theta) * pygsti.sigmaz)
    return expm(generator)

def make_pygsti_model(xstate, gate_depolarization=0, spam_depolarization=0):
    alpha = xstate[0]
    theta = xstate[1]
    epsilon = xstate[2]
    zunitary = make_noisy_z_unitary(alpha)
    xunitary = make_noisy_x_unitary(theta, epsilon)
    model = smq1Q_XYZI.target_model()
    model['Gxpi2', 0] = pygsti.tools.unitary_to_superop(xunitary)
    model['Gzpi2', 0] = pygsti.tools.unitary_to_superop(zunitary)
    model['Gxpi2', 0].depolarize(gate_depolarization)
    model['Gzpi2', 0].depolarize(gate_depolarization)
    model['rho0'].depolarize(spam_depolarization)
    model['Mdefault'].depolarize(spam_depolarization)
    return model


In [15]:
def make_x_sequences(depths):
    cos_circs = []
    sin_circs = []
    for d in depths:
        cos_circs.append(Circuit([('Gxpi2',0 )])*d)
        sin_circs.append(Circuit([('Gxpi2', 0)])*(d+1))
    return cos_circs, sin_circs

def make_z_sequences(depths):
    cos_circs = []
    sin_circs = []
    Gy = Circuit([('Gzpi2', 0), ('Gzpi2', 0), ('Gzpi2', 0), ('Gxpi2', 0), ('Gzpi2', 0)])
    Gy_dagger = Circuit([('Gzpi2', 0), ('Gxpi2', 0), ('Gzpi2', 0), ('Gzpi2', 0), ('Gzpi2', 0)])
    for d in depths:
        cos_circs.append(Gy+Circuit([('Gzpi2',0 )])*d+Gy_dagger)
        sin_circs.append(Gy+Circuit([('Gzpi2', 0)])*d+Circuit([('Gxpi2', 0)]))
    return cos_circs, sin_circs

def make_interleaved_sequences(depths):
    cos_circs = []
    sin_circs = []
    for d in depths:
        cos_circs.append(Circuit([('Gzpi2', 0), ('Gzpi2', 0), ('Gzpi2', 0), ('Gxpi2', 0), ('Gzpi2', 0)])*d)
        sin_circs.append(Circuit([('Gzpi2', 0), ('Gzpi2', 0), ('Gzpi2', 0), ('Gxpi2', 0), ('Gzpi2', 0)])+Circuit([('Gxpi2', 0)]))
    return cos_circs, sin_circs

def make_interleaved_sequences(depths):
    pass

In [16]:
xerror = [0.1, 0.1, 0]
model = make_pygsti_model(xerror, 0.0001, 0.001)

In [17]:
depths = [2**i for i in range(10)]
print(depths)
x_cos_circs, x_sin_circs = make_x_sequences(depths)
z_cos_circs, z_sin_circs = make_z_sequences(depths)
interleaved_cos_circs, 


[1, 2, 4, 8, 16, 32, 64, 128, 256, 512]


In [18]:
all_circs = set(x_cos_circs + x_sin_circs + z_cos_circs + z_sin_circs)
dataset = pygsti.data.simulate_data(model, all_circs, num_samples=1000, seed=0)

In [26]:
def process_zgate_data(dataset, cos_circs, sin_circs, depths):
    experiment = Q()
    for idx, d in enumerate(depths):
        experiment.process_cos(d, (int(dataset[cos_circs[idx]]['0']), int(dataset[cos_circs[idx]]['1'])))
        experiment.process_sin(d, (int(dataset[sin_circs[idx]]['0']), int(dataset[sin_circs[idx]]['1'])))
    analysis = RobustPhaseEstimation(experiment)
    last_good_generation = analysis.check_unif_local(historical=True)
    estimates = analysis.angle_estimates
    return estimates, last_good_generation

def process_xgate_data(dataset, cos_circs, sin_circs, depths):
    experiment = Q()
    for idx, d in enumerate(depths):
        experiment.process_cos(d, (int(dataset[cos_circs[idx]]['0']), int(dataset[cos_circs[idx]]['1'])))
        experiment.process_sin(d, (int(dataset[sin_circs[idx]]['1']), int(dataset[sin_circs[idx]]['0'])))
    analysis = RobustPhaseEstimation(experiment)
    last_good_generation = analysis.check_unif_local(historical=True)
    estimates = analysis.angle_estimates
    return estimates, last_good_generation

def process_interleaved_data(dataset, cos_circs, sin_circs, depths):
    experiment = Q()
    for idx, d in enumerate(depths):
        experiment.process_cos(d, (int(dataset[cos_circs[idx]]['0']), int(dataset[cos_circs[idx]]['1'])))
        experiment.process_sin(d, (int(dataset[sin_circs[idx]]['1']), int(dataset[sin_circs[idx]]['0'])))
    analysis = RobustPhaseEstimation(experiment)
    last_good_generation = analysis.check_unif_local(historical=True)
    estimates = analysis.angle_estimates
    return estimates, last_good_generation

In [27]:
dataset[x_cos_circs[0]]['0']

472.0

In [28]:
process_zgate_data(dataset, z_cos_circs, z_sin_circs, depths)

(array([1.85396081, 1.72895793, 1.71478162, 1.69290503, 1.68648103,
        1.67559477, 1.67209938, 1.67175869, 1.67130961, 1.67113542]),
 9)

In [30]:
process_xgate_data(dataset, x_cos_circs, x_sin_circs, depths)

(array([1.62730177, 1.72160585, 1.70247229, 1.67942154, 1.66770864,
        1.67372245, 1.67118378, 1.67161156, 1.67086767, 1.67087722]),
 9)

In [ ]:
process_interleaved_data(da)

In [29]:
np.pi/2 + xerror[0]

1.6707963267948966

In [24]:
np.pi/2 + xerror[1]

1.6707963267948966

In [25]:
np.pi/(2*depths[-1])

0.0030679615757712823